# 31. 异常设计（Exception Design）

异常设计的目标是：让错误可区分、可处理、可诊断。通常定义自定义基类与子类，在系统边界统一捕获并转成用户可理解的错误。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 28-30 节：异常基础/链/对象


## 知识点地图

- 1. 自定义异常：让错误更语义化
- 2. 异常层级：基类统一捕获，子类细分场景
- 3. 错误边界：在 I/O 边界捕获，在核心逻辑抛出
- 4. 异常携带上下文：必要时加字段


## 自检清单（学完打勾）

- [ ] 会定义自定义异常类（继承 Exception）
- [ ] 会设计异常层级（基类 + 子类）
- [ ] 理解“边界捕获、核心抛出”的原则
- [ ] 异常信息包含足够上下文（但不过度泄露）


## 知识点 1：自定义异常：让错误更语义化

用自定义异常区分业务错误与系统错误，提升可处理性。


In [ ]:
class AppError(Exception):
    pass

class NotFoundError(AppError):
    pass

raise NotFoundError('user not found')


## 知识点 2：异常层级：基类统一捕获，子类细分场景

边界层可捕获 AppError 做统一处理；内部逻辑抛具体子类。


In [ ]:
class AppError(Exception):
    pass

class ValidationError(AppError):
    pass

class NotFoundError(AppError):
    pass


def get_user(users, user_id):
    if user_id not in users:
        raise NotFoundError(f'user {user_id} not found')
    return users[user_id]

users = {1: {'name': 'Ada'}}
try:
    get_user(users, 2)
except AppError as e:
    print('handled:', type(e).__name__, e)


## 知识点 3：错误边界：在 I/O 边界捕获，在核心逻辑抛出

核心逻辑专注正确性；边界层（API/CLI）负责把异常转换为返回码/错误响应。


## 知识点 4：异常携带上下文：必要时加字段

可以在异常对象上保存额外信息（如 user_id），方便日志与处理。


In [ ]:
class NotFoundError(Exception):
    def __init__(self, message, *, user_id=None):
        super().__init__(message)
        self.user_id = user_id

try:
    raise NotFoundError('missing', user_id=123)
except NotFoundError as e:
    print(e, e.user_id)


## 常见坑

- 异常类过多会增加心智负担（只为有处理差异的场景建类）
- 用 Exception 直接表示业务错误会不清晰


## 综合小案例：实现 ConfigError：读取配置失败的统一异常

读取 JSON 配置文件，失败时统一抛 ConfigError（内部可包裹原异常）。


In [ ]:
from pathlib import Path
ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())
import json

class ConfigError(Exception):
    pass


def load_config(path):
    try:
        text = Path(path).read_text(encoding='utf-8')
        return json.loads(text)
    except Exception as e:
        raise ConfigError(f'failed to load config: {path}') from e

p = ART / 'cfg.json'
p.write_text('{bad json}', encoding='utf-8')
try:
    load_config(p)
except ConfigError as e:
    print(type(e).__name__, e)
    print('cause:', type(e.__cause__).__name__)


## 自测题（不写代码也能回答）

- 为什么要设计异常层级？
- “边界捕获、核心抛出”是什么意思？
- 什么时候应该自定义异常类？


## 练习题（建议写代码）

- 设计一个 ValidationError，包含 field 与 message 字段，并演示捕获。
- 实现 parse_user_id(s)：错误时抛 ValidationError（带 user_id 原始字符串）。
